In [0]:
!pip install geopy

Collecting geopy
 Downloading geopy-2.1.0-py3-none-any.whl (112 kB)
███ | 10 kB 12.0 MB/s eta 0:00:01 |█████▉ | 20 kB 11.3 MB/s eta 0:00:01 |████████▊ | 30 kB 6.1 MB/s eta 0:00:01 |███████████▊ | 40 kB 5.0 MB/s eta 0:00:01 |██████████████▋ | 51 kB 5.3 MB/s eta 0:00:01 |█████████████████▌ | 61 kB 5.3 MB/s eta 0:00:01 |████████████████████▍ | 71 kB 5.8 MB/s eta 0:00:01 |███████████████████████▍ | 81 kB 6.1 MB/s eta 0:00:01 |██████████████████████████▎ | 92 kB 5.8 MB/s eta 0:00:01 |█████████████████████████████▏ | 102 kB 5.5 MB/s eta 0:00:01 |████████████████████████████████| 112 kB 5.5 MB/s 
Collecting geographiclib<2,>=1.49
 Downloading geographiclib-1.50-py3-none-any.whl (38 kB)
Installing collected packages: geographiclib, geopy
Successfully installed geographiclib-1.50 geopy-2.1.0
 WARNING: You are using pip version 20.2.4; however, version 21.1.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
from pyspark.sql.functions import udf
from geopy.distance import great_circle 
import datetime as dt
from pyspark.sql.types import TimestampType
from pyspark.sql import functions as F

In [0]:
df_posicoes =  spark.read.format("delta").load("dbfs:/FileStore/tables/batch/bronze/posicoes")
df_posicoes.show(5)

+--------+--------------------+----------+-----------+-----------+-------+
 placa| data_posicao|velocidade| longitude| latitude|ignicao|
+--------+--------------------+----------+-----------+-----------+-------+
TESTE001|Wed Dec 12 2018 0...| 0| -51.469891|-25.3649141| false|
TESTE001|Wed Dec 12 2018 0...| 0|-51.4699098|-25.3649175| false|
TESTE001|Wed Dec 12 2018 0...| 0|-51.4699588|-25.3649551| false|
TESTE001|Wed Dec 12 2018 0...| 0|-51.4698871|-25.3649138| false|
TESTE001|Wed Dec 12 2018 0...| 0|-51.4699223| -25.364885| false|
+--------+--------------------+----------+-----------+-----------+-------+
only showing top 5 rows

In [0]:
df_pois =  spark.read.format("delta").load("dbfs:/FileStore/tables/batch/bronze/pois")
df_pois.show(5)

+-------+----+------------------+-------------------+
 nome|raio| latitude| longitude|
+-------+----+------------------+-------------------+
PONTO 1| 300|-25.56742701740896| -51.47653363645077|
PONTO 2| 300| -25.568056| -51.480278|
PONTO 3| 250| -25.414167| -51.566944|
PONTO 4| 250| -25.718611| -51.831111|
PONTO 5| 163|-25.37240459807051|-51.497342622606084|
+-------+----+------------------+-------------------+
only showing top 5 rows

In [0]:
def get_poi(long, lat, line_number = 0):
  try:
    row = rows[line_number]
  except:
    return ''

  if great_circle((long, lat), (row['longitude'], row['latitude']), ).meters <= int(row['raio']):
    return row['nome']
  
  return get_poi(long, lat, line_number + 1)

In [0]:
rows = df_pois.collect()

get_poi_udf = udf(get_poi)

df_posicoes = df_posicoes.withColumn('poi', get_poi_udf('longitude', 'latitude'))
df_posicoes.show(5)

+--------+--------------------+----------+-----------+-----------+-------+--------+
 placa| data_posicao|velocidade| longitude| latitude|ignicao| poi|
+--------+--------------------+----------+-----------+-----------+-------+--------+
TESTE001|Wed Dec 12 2018 0...| 0| -51.469891|-25.3649141| false|PONTO 24|
TESTE001|Wed Dec 12 2018 0...| 0|-51.4699098|-25.3649175| false|PONTO 24|
TESTE001|Wed Dec 12 2018 0...| 0|-51.4699588|-25.3649551| false|PONTO 24|
TESTE001|Wed Dec 12 2018 0...| 0|-51.4698871|-25.3649138| false|PONTO 24|
TESTE001|Wed Dec 12 2018 0...| 0|-51.4699223| -25.364885| false|PONTO 24|
+--------+--------------------+----------+-----------+-----------+-------+--------+
only showing top 5 rows

In [0]:
get_date_udf = udf(lambda date: dt.datetime.strptime(date.replace(" (Hora oficial do Brasil)", ""), '%a %b %d %Y %H:%M:%S GMT-0200'), TimestampType())

df_posicoes = df_posicoes.withColumn('data_posicao', get_date_udf('data_posicao'))
df_posicoes.show(5)

+--------+-------------------+----------+-----------+-----------+-------+--------+
 placa| data_posicao|velocidade| longitude| latitude|ignicao| poi|
+--------+-------------------+----------+-----------+-----------+-------+--------+
TESTE001|2018-12-12 00:04:03| 0| -51.469891|-25.3649141| false|PONTO 24|
TESTE001|2018-12-12 00:34:06| 0|-51.4699098|-25.3649175| false|PONTO 24|
TESTE001|2018-12-12 01:04:09| 0|-51.4699588|-25.3649551| false|PONTO 24|
TESTE001|2018-12-12 01:34:12| 0|-51.4698871|-25.3649138| false|PONTO 24|
TESTE001|2018-12-12 02:04:15| 0|-51.4699223| -25.364885| false|PONTO 24|
+--------+-------------------+----------+-----------+-----------+-------+--------+
only showing top 5 rows

In [0]:
df_posicoes = df_posicoes.withColumn('velocidade', df_posicoes.velocidade.cast('int'))
df_posicoes = df_posicoes.withColumn('status', F.when((F.col('ignicao') == 'false') & (F.col('velocidade') < 5), 0).otherwise(1))
df_posicoes.show(5)

+--------+-------------------+----------+-----------+-----------+-------+--------+------+
 placa| data_posicao|velocidade| longitude| latitude|ignicao| poi|status|
+--------+-------------------+----------+-----------+-----------+-------+--------+------+
TESTE001|2018-12-12 00:04:03| 0| -51.469891|-25.3649141| false|PONTO 24| 0|
TESTE001|2018-12-12 00:34:06| 0|-51.4699098|-25.3649175| false|PONTO 24| 0|
TESTE001|2018-12-12 01:04:09| 0|-51.4699588|-25.3649551| false|PONTO 24| 0|
TESTE001|2018-12-12 01:34:12| 0|-51.4698871|-25.3649138| false|PONTO 24| 0|
TESTE001|2018-12-12 02:04:15| 0|-51.4699223| -25.364885| false|PONTO 24| 0|
+--------+-------------------+----------+-----------+-----------+-------+--------+------+
only showing top 5 rows

In [0]:
df_posicoes.write.mode("overwrite").format("delta").save("dbfs:/FileStore/tables/batch/silver/frota/")